In [ ]:
#!pip install transformers datasets torch accelerate
#!pip install sentencepiece
#!pip install --upgrade datasets evaluate


In [1]:
import json
from datasets import Dataset, load_from_disk
data_train_path = "./data/train"
data_train = []
with open(data_train_path, "r") as file:
    for line in file:
        example = json.loads(line.strip())
        data_train.append({
            "input_text": example["src"],
            "target_text": example["dst"]
        })

data_val_path = "./data/val"
data_val = []
with open(data_val_path, "r") as file:
    for line in file:
        example = json.loads(line.strip())
        data_val.append({
            "input_text": example["src"],
            "target_text": example["dst"]
        })

print(data_train[:1])
print(data_val[:1])
for i in data_train[:20]:
    print(i['target_text'])

/home/mike/anaconda3/envs/nlp_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'input_text': '◄▴◓◠▨ ◨▽◠▦◈◬◓▪▼◬▵', 'target_text': '- Intriguing.'}]
[{'input_text': "◘◚ ◞◠▷◫◀◗ ▫◠▨◬◎ ▨◪▦◈◫▦◫ ▫◧▻▱◠◈▪ ◚◪ ◝◂▾▼▷◠◓◈'◬▦ 27:37'◈▴▨◗ ◕◂▱◭ ◀◗◓ ▨▴▢ ◈◠▷◠ ◞▨◂◓◨ ▴◒◗▫▱◪◈◗▵", 'target_text': 'The hosts regrouped, and Bouchard evened the score again, scoring a goal with a 27-37 man advantage.'}]
- Intriguing.
He would need to repeat his vows in the land of the living and drink from the wine of ages.
You couldn't even answer my texts?
How fast do you go?
He's talking about a few right here in Lisbon, mostly Jews who have escaped the Germans, that's all.
For instance, Ben is not a guy who skips work.
Goodbye, Jakob.
Sure beats the foam domes round here.
Oh, wait a second!
The time is now.
Our creditors are voting with their pocketbook by buying up our debt.
Over there.
She-- she's the purple sandpiper.
I'm meeting a friend.
To ensure peace between the two royal households.
I worked even if the boss kicked or slapped me.
I could eat even you, Super Mario.
But there's 12 of them.
She's

In [2]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

ds_train = Dataset.from_list(data_train)
ds_val = Dataset.from_list(data_val)
def preprocess_data(examples):
    model_inputs = tokenizer(
        examples["input_text"], max_length=128, truncation=True, padding="max_length"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["target_text"], max_length=128, truncation=True, padding="max_length"
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_ds_train = ds_train.map(preprocess_data, batched=True)
tokenized_ds_val = ds_val.map(preprocess_data, batched=True)
#print(tokenized_ds_train[0])
tokenized_ds_train.save_to_disk("tokenized_ds_train_small")
tokenized_ds_val.save_to_disk("tokenized_ds_val_small")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Map:   0%|                                                                                      | 0/300000 [00:00<?, ? examples/s]/home/mike/anaconda3/envs/nlp_env/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  

KeyboardInterrupt: 

In [1]:
from datasets import Dataset, load_from_disk
tokenized_ds_train = load_from_disk("tokenized_ds_train")
tokenized_ds_val = load_from_disk("tokenized_ds_val").train_test_split(test_size=0.1)["test"]

/home/mike/anaconda3/envs/nlp_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
len(tokenized_ds_val)

50

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

model_name = "./t5-finetuned_new/checkpoint-500"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print('Done')

Done


In [4]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-finetuned_new",       # Папка для сохранения модели
    learning_rate=5e-4,               # Начальная скорость обучения
    per_device_train_batch_size=13,   # Размер батча для обучения
    per_device_eval_batch_size=13,    # Размер батча для оценки
    num_train_epochs=5,               # Количество эпох
    weight_decay=0.01,                # Регуляризация (L2)
    save_total_limit=20,              # Сохранение только последних двух чекпоинтов
    logging_dir="./logs",             # Папка для логов
    logging_steps=10,                 # Частота логирования
    save_steps=100,                  # Частота сохранения модели
    fp16=False,                        # Использование 16-битной точности (ускоряет обучение на GPU)
    report_to="none",                 # Отключение логирования в WandB или Hugging Face Hub
    #lr_scheduler_type="linear",       # Linear Scheduler
    warmup_steps=50                  # Количество шагов разогрева
)

In [5]:
import sacrebleu
def compute_bleu(eval_pred):
    predictions, labels = eval_pred

    # Decode predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Decode labels (references)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Format references for SacreBLEU
    formatted_labels = [[label] for label in decoded_labels]  # Each reference must be a list of strings

    # Compute BLEU
    result = sacrebleu.corpus_bleu(decoded_preds, formatted_labels)

    return {"bleu": result.score}
    
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds_train,
    eval_dataset=tokenized_ds_val,
    processing_class=tokenizer,
    compute_metrics=compute_bleu
)

In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,0.456600
20,0.567800
30,0.437300
40,0.451300
50,0.568100
60,0.438800
70,0.403600
80,0.451800
90,0.461500
100,0.449100


In [2]:
import json

input_file = "./data/test_no_reference"

data = []
with open(input_file, "r") as file:
    for line in file:
        record = json.loads(line.strip())
        data.append(record)  # Каждая строка — это словарь с ключом "src"

print(f"Загружено {len(data)} записей")

Загружено 1000 записей


In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_path = "./t5-finetuned_new/checkpoint-1500"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def translate_text(src_text):
    inputs = tokenizer(src_text, return_tensors="pt", max_length=128, truncation=True).to(device)
    outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [4]:
from tqdm import tqdm
for record in tqdm(data_train, desc="Перевод текстов"):
    #print(record)
    src_text = record["input_text"]
    tmp = translate_text(src_text) 
    print(record['target_text'])
    print(tmp)

Перевод текстов:   0%|                                                   | 1/300000 [00:01<118:12:03,  1.42s/it]

- Intriguing.
I'm sorry.


Перевод текстов:   0%|                                                    | 2/300000 [00:02<79:32:15,  1.05it/s]

He would need to repeat his vows in the land of the living and drink from the wine of ages.
I'm going to get a few weeks, and I'm going to get a few weeks.


Перевод текстов:   0%|                                                    | 4/300000 [00:02<36:54:10,  2.26it/s]

...

Перевод текстов:   0%|                                                  | 138/300000 [01:06<41:37:37,  2.00it/s]

I don't want to some guy staying here masturbating all over the place.
I'm going to get a few weeks, and I'm going to get a few weeks.


Перевод текстов:   0%|                                                  | 139/300000 [01:06<39:53:25,  2.09it/s]

Honey, are you all right?
What are you doing?


KeyboardInterrupt: 

In [6]:
from tqdm import tqdm
for record in tqdm(data, desc="Перевод текстов"):
    src_text = record["src"]
    record["dst"] = translate_text(src_text) 

Перевод текстов: 100%|██████████████████████████████████████████████████████| 1000/1000 [12:51<00:00,  1.30it/s]


In [7]:
output_file = "data/answ3.jsonl"
with open(output_file, "w") as file:
    for record in data:
        file.write(json.dumps(record, ensure_ascii=False) + "\n")

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def compute_bleu_nltk(predictions, references):
    smoothing = SmoothingFunction().method1
    bleu_scores = [
        sentence_bleu([ref.split()], pred.split(), smoothing_function=smoothing)
        for pred, ref in zip(predictions, references)
    ]
    return sum(bleu_scores) / len(bleu_scores)